In [1]:
# https://pubs.acs.org/doi/10.1021/acs.jctc.2c01057

import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from numpy import linalg as LA

from qiskit import transpile
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from IPython.display import clear_output
from scipy.sparse.linalg import eigs
from scipy import sparse
import math
import time
from joblib import Parallel, delayed
import multiprocessing as multip
from qiskit.quantum_info import Statevector






In [2]:


"""circuit construction"""
num_q = 4
layer = 1
num_p = num_q*layer 
weights = ParameterVector("weight",num_p)



def circuit_HEA(weights):
    circ = QuantumCircuit(num_q, num_q)
    
    for i in range(layer):
        for j in range(num_q):
            circ.ry(weights[num_q*i+j], j)
        for j in range(num_q-1):
            circ.cz(j, j+1)
    
    return circ 




print(circuit_HEA(np.zeros(num_p)))


simulator = AerSimulator()

def estimate_ZZ(WEIGHT,  Z_1_index, Z_2_index, SHOTS):
    estimate = 0
    qc = circuit_HEA(WEIGHT)
    qc = transpile(qc, simulator)
    ind = list(range(num_q))
    rind = ind
    rind.reverse()
    qc.measure(ind, rind)
    result = simulator.run(qc, shots = SHOTS, memory=True).result()
    c = result.get_memory(qc) ## output distribution of 0 and 1
    for i in range(SHOTS):
        if c[i][num_q-1-Z_1_index] == c[i][num_q-1-Z_2_index]:
            estimate += 1
        else:
            estimate += -1
    estimate = estimate/SHOTS
    
    return estimate

def estimate_Z(WEIGHT, Z_index, SHOTS):
    estimate = 0
    qc = circuit_HEA(WEIGHT)
    qc = transpile(qc, simulator)
    ind = list(range(num_q))
    rind = ind
    rind.reverse()
    qc.measure(ind, rind)
    result = simulator.run(qc, shots = SHOTS, memory=True).result()
    c = result.get_memory(qc) ## output distribution of 0 and 1
    for i in range(SHOTS):
        if c[i][num_q-1-Z_index] == '0':
            estimate += 1
        else:
            estimate += -1
    
    estimate = estimate/SHOTS
    
    return estimate


     ┌───────┐         
q_0: ┤ Ry(0) ├─■───────
     ├───────┤ │       
q_1: ┤ Ry(0) ├─■──■────
     ├───────┤    │    
q_2: ┤ Ry(0) ├────■──■─
     ├───────┤       │ 
q_3: ┤ Ry(0) ├───────■─
     └───────┘         
c: 4/══════════════════
                       


In [3]:
shots = 10000
weights = np.random.normal(0,4*np.pi**2,num_p)



# example1: H = ZIII  index = 3

H = SparsePauliOp(['ZIII'], 1)
Hmat = Operator(H)
circ = circuit_HEA(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)

print('error between expectation and measurement result:',np.abs(estimate_Z(weights, 3, shots) - true_value))

# example2: H = IZII index = 2

H = SparsePauliOp(['IZII'], 1)
Hmat = Operator(H)
circ = circuit_HEA(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)


print('error between expectation and measurement result:',np.abs(estimate_Z(weights, 2, shots) - true_value))

# example3: H = ZZII index = 3,2

H = SparsePauliOp(['ZZII'], 1)
Hmat = Operator(H)
circ = circuit_HEA(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)


print('error between expectation and measurement result:',np.abs(estimate_ZZ(weights, 3,2, shots) - true_value))
 
# example4: H = ZIZI index = 3,1

H = SparsePauliOp(['ZIZI'], 1)
Hmat = Operator(H)
circ = circuit_HEA(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)


print('error between expectation and measurement result:',np.abs(estimate_ZZ(weights, 3, 1, shots) - true_value))


error between expectation and measurement result: 0.004153983533212724
error between expectation and measurement result: 0.008462390229495464
error between expectation and measurement result: 0.004427229179205239
error between expectation and measurement result: 0.014558665079883265


In [4]:


"""circuit construction"""
num_q = 4
layer = 1
num_p = 4*layer
weights = ParameterVector("weight",num_p)


def circuit_QAOA_XXZ(weights):
    circ = QuantumCircuit(num_q, num_q)
    
    ## input layer to prepare bell state -\psi
    for j in range(num_q):
        circ.x(j)
    for j in range(int(num_q/2)):
        circ.h(2*j)
        circ.cx(2*j,2*j+1)
    
    ## ansatz with alternating 2-local qubit rotation gates
    
    for i in range(layer):
        ## odd layers
        for j in range(int(num_q/2)):
            circ.rzz(weights[4*i],2*j+1,(2*j+2) % num_q) ## ZZ gates in odd sum
        for j in range(int(num_q/2)):
            circ.ryy(weights[4*i+1],2*j+1,(2*j+2) % num_q) ## YY gates in odd sum
        for j in range(int(num_q/2)):
            circ.rxx(weights[4*i+1],2*j+1,(2*j+2) % num_q) ## XX gates in odd sum
        
        ## even layers
        for j in range(int(num_q/2)):
            circ.rzz(weights[4*i+2],2*j,2*j+1) ## ZZ gates in even sum
        for j in range(int(num_q/2)):
            circ.ryy(weights[4*i+3],2*j,2*j+1) ## YY gates in even sum
        for j in range(int(num_q/2)):
            circ.rxx(weights[4*i+3],2*j,2*j+1) ## XX gates in even sum

        
    return circ 

print(circuit_QAOA_XXZ(np.zeros(num_p)))


simulator = AerSimulator()

def estimate_XX_QAOA(WEIGHT,  X_1_index, X_2_index, SHOTS):
    estimate = 0
    qc = circuit_QAOA_XXZ(WEIGHT)
    for i in range(num_q):
        qc.h(i)
    qc = transpile(qc, simulator)
    ind = list(range(num_q))
    rind = ind
    rind.reverse()
    qc.measure(ind, rind)
    result = simulator.run(qc, shots = SHOTS, memory=True).result()
    c = result.get_memory(qc) ## output distribution of 0 and 1
    for i in range(SHOTS):
        if c[i][num_q-1-X_1_index] == c[i][num_q-1-X_2_index]:
            estimate += 1
        else:
            estimate += -1
    estimate = estimate/SHOTS
    
    return estimate

def estimate_YY_QAOA(WEIGHT,  Y_1_index, Y_2_index, SHOTS):
    estimate = 0
    qc = circuit_QAOA_XXZ(WEIGHT)
    for i in range(num_q):
        qc.sdg(i)
        qc.h(i)
    qc = transpile(qc, simulator)
    ind = list(range(num_q))
    rind = ind
    rind.reverse()
    qc.measure(ind, rind)
    result = simulator.run(qc, shots = SHOTS, memory=True).result()
    c = result.get_memory(qc) ## output distribution of 0 and 1
    for i in range(SHOTS):
        if c[i][num_q-1-Y_1_index] == c[i][num_q-1-Y_2_index]:
            estimate += 1
        else:
            estimate += -1
    estimate = estimate/SHOTS
    
    return estimate

def estimate_ZZ_QAOA(WEIGHT,  Z_1_index, Z_2_index, SHOTS):
    estimate = 0
    qc = circuit_QAOA_XXZ(WEIGHT)
    qc = transpile(qc, simulator)
    ind = list(range(num_q))
    rind = ind
    rind.reverse()
    qc.measure(ind, rind)
    result = simulator.run(qc, shots = SHOTS, memory=True).result()
    c = result.get_memory(qc) ## output distribution of 0 and 1
    for i in range(SHOTS):
        if c[i][num_q-1-Z_1_index] == c[i][num_q-1-Z_2_index]:
            estimate += 1
        else:
            estimate += -1
    estimate = estimate/SHOTS
    
    return estimate


     ┌───┐┌───┐                                ┌─────────┐           »
q_0: ┤ X ├┤ H ├──■───────────■─────────────────┤1        ├───────────»
     ├───┤└───┘┌─┴─┐         │      ┌─────────┐│         │┌─────────┐»
q_1: ┤ X ├─────┤ X ├─■───────┼──────┤0        ├┤         ├┤0        ├»
     ├───┤┌───┐└───┘ │ZZ(0)  │      │  Ryy(0) ││  Ryy(0) ││  Rxx(0) │»
q_2: ┤ X ├┤ H ├──■───■───────┼──────┤1        ├┤         ├┤1        ├»
     ├───┤└───┘┌─┴─┐         │ZZ(0) └─────────┘│         │└─────────┘»
q_3: ┤ X ├─────┤ X ├─────────■─────────────────┤0        ├───────────»
     └───┘     └───┘                           └─────────┘           »
c: 4/════════════════════════════════════════════════════════════════»
                                                                     »
«     ┌─────────┐        ┌─────────┐┌─────────┐
«q_0: ┤1        ├─■──────┤0        ├┤0        ├
«     │         │ │ZZ(0) │  Ryy(0) ││  Rxx(0) │
«q_1: ┤         ├─■──────┤1        ├┤1        ├
«     │  Rxx(0) │        ├─

In [5]:
shots = 10000
weights = np.random.normal(0,4*np.pi**2,num_p)



# example1: H = XXII index = 3,2

H = SparsePauliOp(['XXII'], 1)
Hmat = Operator(H)
circ = circuit_QAOA_XXZ(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)

print('error between expectation and measurement result:',np.abs(estimate_XX_QAOA(weights, 3,2, shots) - true_value))

# example2: H = IYYI index = 2,1

H = SparsePauliOp(['IYYI'], 1)
Hmat = Operator(H)
circ = circuit_QAOA_XXZ(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)


print('error between expectation and measurement result:',np.abs(estimate_YY_QAOA(weights, 2,1, shots) - true_value))

# example3: H = ZIIZ index = 3,0
 
H = SparsePauliOp(['ZIIZ'], 1)
Hmat = Operator(H)
circ = circuit_QAOA_XXZ(weights)
psi = np.array(Statevector(circ))
Hpsi = Hmat.dot(psi)
true_value = np.inner(np.conjugate(psi),Hpsi)


print('error between expectation and measurement result:',np.abs(estimate_ZZ_QAOA(weights, 3,0, shots) - true_value))


error between expectation and measurement result: 0.001608319264451319
error between expectation and measurement result: 0.005422718887395317
error between expectation and measurement result: 0.00232417533284901
